In [ ]:
#########################################################################
# 모델을 사전학습할 경우, 실제는 더 많은 데이터가 사용되기는 하지만, 유사한 순서로 진행됨
########################################################################

In [1]:
from datasets import load_dataset
dataset = load_dataset("klue","ynat")
dataset['train'][0]

{'guid': 'ynat-v1_train_00000',
 'title': '유튜브 내달 2일까지 크리에이터 지원 공간 운영',
 'label': 3,
 'url': 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=227&oid=001&aid=0008508947',
 'date': '2016.06.30. 오전 10:36'}

In [2]:
# train과 vailidation 데이터셋 title을 줄바꿈 단위로 파일에 저장. 
# 메모리를 낭비하지 않기 위해 파일 저장 후 불러와 쓰는 습관을 들이는 것이 좋음

target_key = "title"
for key in dataset.column_names.keys():
    with open(f"/home/ubuntu/model_path/tokenizer_data_{key}.txt", "w") as f:
        f.write("\n".join(dataset[key][target_key]))

In [ ]:
##############################################################################
#
#  Tokenizer Training
#
##############################################################################

In [3]:
# 특수 토큰 정의 - BERT모델에서 주로 사용하는 특수 토큰을 포함하도록 실습
# 토크나이저 학습중에는 사용하지 않고 모델의 기술적 부분을 위해 필요

user_defined_symbols = [
    "[PAD]", # 문장 길이를 맞추기 위해 사용
    "[UNK]", # 토크나이저가 인식할 수 없는 토큰
    "[CLS]", # BERT 계열에서 문장 전체 정보를 저장하는 토큰
    "[SEP]", # BERT 계열에서 문장 구분을 위해 사용하는 토큰
    "[MASK]", # Masked LM에서 토큰 마스킹을 위해 사용하는 토큰
]

unused_token_num = 100
unused_list = [f"[UNUSED{i}]" for i in range(100)] # 사전학습시 어휘에 없는 토큰 추가를 위한 빈공간

whole_user_defined_symbols = user_defined_symbols + unused_list

print(whole_user_defined_symbols[:10])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '[UNUSED0]', '[UNUSED1]', '[UNUSED2]', '[UNUSED3]', '[UNUSED4]']


In [6]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece

# 우선 transformer 라이브러리 대신 tokenizer 라이브러리 사용
# WordPiece라는 규칙만 지정된, 빈 상태의 토크나이저 선언
bert_tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))

# 정규화 진행
from tokenizers import normalizers
normalizer = normalizers.BertNormalizer()
bert_tokenizer.normalizer = normalizer

normalizer.normalize_str("Hell how\nare u? ")

'hell how are u? '

In [7]:
# Wordpiece 이전에 먼저 적용되는 토크나이저 
from tokenizers.pre_tokenizers import Whitespace
pre_tokenizer = Whitespace()
bert_tokenizer.pre_tokenizer = pre_tokenizer
pre_tokenizer.pre_tokenize_str("안녕하세요. 제대로 인코딩이 되는지 확인 중입니다.")

[('안녕하세요', (0, 5)),
 ('.', (5, 6)),
 ('제대로', (7, 10)),
 ('인코딩이', (11, 15)),
 ('되는지', (16, 19)),
 ('확인', (20, 22)),
 ('중입니다', (23, 27)),
 ('.', (27, 28))]

In [8]:
# 문장이 인코딩되었을 때 취해야 할 형식 작성
# BERT 모델이 요구하는 형식 - 가장 앞에 [CLS] 토큰이 있어야 하고, 문장 구별을 위해서는 [SEP] 토큰이 사용됨

from tokenizers.processors import TemplateProcessing

post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A $B:1 [SEP]:1",
    special_tokens=[(t,i) for i,t in enumerate(user_defined_symbols)],
)

bert_tokenizer.post_processor=post_processor

In [9]:
# 토크나이저 학습을 위한 trainer 생성

from tokenizers.trainers import WordPieceTrainer

vocab_size = 24000
trainer = WordPieceTrainer(
    vocab_size=vocab_size,
    special_tokens=whole_user_defined_symbols,
)

In [11]:
# 학습 수행
from glob import glob

bert_tokenizer.train(glob(f"/home/ubuntu/model_path/*.txt"), trainer)

In [13]:
# 학습 결과 확인

output = bert_tokenizer.encode("인코딩 및 디코딩이 학습 후 제대로 되는지 확인합니다.")
print(output.ids)

bert_tokenizer.decode(output.ids) # 디코딩 방법이 적용되지 않았으므로 #등의 문자가 나오는 정상적인 오류 상태가 나옴

[2, 675, 906, 2220, 4518, 1240, 906, 2220, 569, 6985, 905, 6727, 6464, 586, 1881, 4308, 106, 3]


'인 ##코 ##딩 및 디 ##코 ##딩 ##이 학습 후 제대로 되는 ##지 확인 ##합니다 .'

In [14]:
# 디코더 설정
from tokenizers import decoders

bert_tokenizer.decoder = decoders.WordPiece()
bert_tokenizer.decode(output.ids)

'인코딩 및 디코딩이 학습 후 제대로 되는지 확인합니다.'

In [15]:
# 학습한 토크나이저를 Transformers 라이브러리로 옮기기
# 토크나이저를 불러올 때 from_pretrained 대신 초기화(__init__)함수를 사용해 생성 - 이때 tokenizer_object 파라미터에 토크나이저 입력

from transformers import BertTokenizerFast

fast_tokenizer = BertTokenizerFast(tokenizer_object=bert_tokenizer)
encoded = fast_tokenizer.encode("인코딩 및 디코딩이 학습 후 제대로 되는지 확인합니다.")
decoded = fast_tokenizer.decode(encoded)

print(encoded)
print(decoded)

[2, 675, 906, 2220, 4518, 1240, 906, 2220, 569, 6985, 905, 6727, 6464, 586, 1881, 4308, 106, 3]
[CLS] 인코딩 및 디코딩이 학습 후 제대로 되는지 확인합니다. [SEP]


In [16]:
# Transformer 라이브러리에서 사용하는 형식대로 모델 저장
output_dir = "/home/ubuntu/model_path"
fast_tokenizer.save_pretrained(output_dir)

('/home/ubuntu/model_path/tokenizer_config.json',
 '/home/ubuntu/model_path/special_tokens_map.json',
 '/home/ubuntu/model_path/vocab.txt',
 '/home/ubuntu/model_path/added_tokens.json',
 '/home/ubuntu/model_path/tokenizer.json')

In [18]:
# 저장한 토크나이저 다시 불러와서 테스트
new_tokenizer = BertTokenizerFast.from_pretrained(output_dir)
encoded = new_tokenizer(["인코딩 잘 되는지 확인", "안되면 다시 처음부터 학습하자"])

for k,v in encoded.items():
    print(k,v)

print(new_tokenizer.decode(encoded["input_ids"][0]))
print(new_tokenizer.decode(encoded["input_ids"][1]))

input_ids [[2, 675, 906, 2220, 1675, 6464, 586, 1881, 3], [2, 18632, 1594, 22573, 6985, 3782, 3]]
token_type_ids [[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]]
attention_mask [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]
[CLS] 인코딩 잘 되는지 확인 [SEP]
[CLS] 안되면 다시 처음부터 학습하자 [SEP]


In [ ]:
##############################################################################
#
#  Model Training
#
##############################################################################

In [20]:
# 데이터셋 및 토크나이저 불러오기
dataset = load_dataset("klue", "ynat")
tokenizer_path = "/home/ubuntu/model_path"
tokenizer = BertTokenizerFast.from_pretrained(tokenizer_path)

In [22]:
# 모델 최초 선언에 필요한 config 준비 
# embedding size, hidden size, num layers등 모델의 전반적 구조 정보가 저장됨
# 다른 정보는 base 모델을 따라가되, vocab 정보는 맞추어줘야 함
from transformers import BertConfig

mycfg=BertConfig(vocab_size=tokenizer.vocab_size)
print(mycfg)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.52.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 24000
}



In [24]:
# config에 따른 모델 선언
from transformers import BertForMaskedLM

model=BertForMaskedLM(mycfg)
print(model.config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.52.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 24000
}

